# Feather File Format In Python | Less Space, More Speed !
".feather" file format is an optimized binary format, really faster to be read by python.

Using this file as input you can download the whole dataset 8 time faster than using "datatable" and almoust 20 time faster than the commonly used read_csv() method.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import numba

# @numba.jit
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
%%time
# Reading the  csv files with pandas.read_csv take 8.78 s.
train_0 = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv").set_index("Id")
test_0 = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv").set_index("Id")
sample_0 = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv').set_index("Id")


In [ ]:

import datatable as dt  # pip install datatable

In [ ]:
%%time
# Reading csv files with datatable takes 3.47 s

train = dt.fread("../input/tabular-playground-series-dec-2021/train.csv").to_pandas().set_index("Id")
test = dt.fread("../input/tabular-playground-series-dec-2021/test.csv").to_pandas().set_index("Id")
sample = dt.fread('../input/tabular-playground-series-dec-2021/sample_submission.csv').to_pandas().set_index("Id")

TARGET = "Cover_Type"

In [ ]:
# Creating Feather file:
train.reset_index().to_feather("Dec21TP_train.feather");
test.reset_index().to_feather("Dec21TP_test.feather");
sample.reset_index().to_feather("Dec21TP_sample.feather");

# To read these files just write

# train = pd.read_feather('../input/nov21tp-feathercreator/Nov21TP_train.feather')

# test = pd.read_feather('../input/nov21tp-feathercreator/Nov21TP_test.feather')
# This operation will take 4 times faster than 

In [ ]:
%%time
# Reading the feather files and creating the same dataset takes 0.539 s!

train_2 = pd.read_feather('../working/Dec21TP_train.feather')
test_2 = pd.read_feather('../working/Dec21TP_test.feather')
sample_2 = pd.read_feather('../working/Dec21TP_sample.feather')
# This operation will take 8 times faster than with datatable files: Just use this file as input of your work

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
ss = StandardScaler()
mms = MinMaxScaler()
rs = RobustScaler()
target = train[TARGET]
train = train.drop(columns=TARGET)

In [ ]:
DATASET = "Dec21"

transforms = {"StandardScaler": ss,
              "MinMaxScalermms": mms, 
              "RobustScaler":rs}
for name, transform_ in transforms.items():
    print(f"Fitting {name}")
    t_train = pd.DataFrame(transform_.fit_transform(train), columns=train.columns, index = train.index)
    print("train done")
    t_test = pd.DataFrame(transform_.transform(test), columns=test.columns, index = test.index)
    print("test done")
    t_train["target"] = target
    train_file_name = DATASET + "_train_" + name + ".feather"
    test_file_name = DATASET + "_test_" + name + ".feather"
    t_train.reset_index().to_feather(train_file_name);
    print(f"{train_file_name} created")
    t_test.reset_index().to_feather(test_file_name);
    print(f"{test_file_name} created")